# Counting Sundays

> You are given the following information, but you may prefer to do some research for yourself.
> * 1 Jan 1900 was a Monday.
> * Thirty days has September,
>   April, June and November.
>   All the rest have thirty-one,
>   Saving February alone,
>   Which has twenty-eight, rain or shine.
>   And on leap years, twenty-nine.
> * A leap year occurs on any year evenly divisible by 4, but not on a century unless it is divisible by 400.
>
> How many Sundays fell on the first of the month during the twentieth century (1 Jan 1901 to 31 Dec 2000)?


I know several methods for determining the day of the week for a given calendar date. I’ll use this as an opportunity to explore Julia modules for namespacing. *Note, given the date range of interest in this problem, I’m going to simplify all of the algorithms to assume dates in the Gregorian calendar.* I’ll start with a module to hold some shared utility code.

In [1]:
module CalendarUtil
export daynames, isleapyear

daynames = Dict(0 => "Sunday", 1 => "Monday", 2 => "Tuesday", 3 => "Wednesday", 4 => "Thursday", 5 => "Friday", 6 => "Saturday")

isleapyear(year) = mod(year, 4) == 0 && (mod(year, 100) != 0 || mod(year, 400) == 0)
end

Main.CalendarUtil

## Astronomers’ Method

In his book, *Astronomical Algorithms*, Jean Meeus gives a method for finding the ‘Julian Day Number’ for a calendar date—the count of days since noon on the beginning of the year -4712. The day of the week can then be found by computing the residual modulo 7.

In [2]:
module Meeus
using Main.CalendarUtil

function julian_day(Y, M, D)
    if M < 3
        Y -= 1
        M += 12
    end
    
    A = Y ÷ 100
    B = 2 - A + A ÷ 4
    
    floor(365.25 * (Y + 4716)) + floor(30.6001 * (M + 1)) + D + B - 1524.5    
end

function dayofweek(year, month, date)
    daynames[mod(Integer(1.5 + julian_day(year, month, date)), 7)]    
end

end

Main.Meeus

## Doomsday Algorithm

This method was devised by John Conway with inspiration from Martin Gardner and Lewis Carroll. It observes that there are easily-remembered dates that all fall on the same day of the week as each other in a given year. It then finds that ‘doomsday’ for a given year, and the offset from an adjacent date to the date of interest.

In [4]:
module Conway
using Main.CalendarUtil

function dayofweek(year, month, date)
    centuryDay = Dict(16 => 2, 17 => 0, 18 => 5, 19 => 3, 20 => 2, 21 => 0)
    doomsday = Dict(3 => 0, 4 => 4, 5 => 9, 6 => 6, 7 => 11, 8 => 8, 9 => 5, 10 => 10, 11 => 7, 12 => 12)
    doomsday[1] = isleapyear(year) ? 4 : 3 # doomsdays for January and February differ in leap years
    doomsday[2] = isleapyear(year) ? 29 : 28
    cc, yy = divrem(year, 100) # 1971 => 19, 71

    thumb = mod(date - doomsday[month], 7) # Doomsday Difference
    index = centuryDay[cc] # Century Day
    middle = div(yy, 12) # Number of Dozens
    ring = rem(yy, 12) # Remainder
    pinkie = div(ring, 4) # Leap Years in Remainder
    
    daynames[mod(thumb + index + middle + ring + pinkie, 7)]
end

end

Main.Conway

## Lewis Carroll’s Method

This method was published as a [short article](https://www.nature.com/articles/035517a0) in Nature in 1887.

In [6]:
module Carroll
using Main.CalendarUtil

function dayofweek(year, month, date)
    cc, yy = divrem(year, 100) # 1971 => 19, 71
    centuryItem = 2 * (3 - rem(cc, 4))
    dozens, overplus = divrem(yy, 12)
    yearItem = dozens + overplus + div(overplus, 4)
    forMonth = Dict(1 => 0, 2 => 3, 3 => 3, 4 => 6, 5 => 1, 6 => 4, 7 => 6, 8 => 2, 9 => 5, 10 => 0, 11 => 3, 12 => 5)
    monthItem = forMonth[month]
    dayItem = date
    total = centuryItem + yearItem + monthItem + dayItem
    if isleapyear(year) && month < 3
        total -= 1
    end
    daynames[mod(total, 7)]
end

end

Main.Carroll

## System Date Library

We can also use the `Dates` module in Julia’s Standard Library for this task.

In [8]:
module SysLib
using Dates

function dayofweek(year, month, date)
    Dates.dayname(Date(year, month, date))
end

end

Main.SysLib

In [9]:
SysLib.dayofweek(1971, 10, 22)

"Friday"

## Results

To find the answer to the given problem, we compute the day of the week for the first day of each month of interest and count how many of them are Sundays. To use all of our implementations, we’ll use a function that takes a ‘day of week’ function as a parameter. They should all give the same result!

In [10]:
function p19(dow)
    length(filter(isequal("Sunday"),[dow(y, m, 1) for y in 1901:2000, m in 1:12]))
end

p19 (generic function with 1 method)

In [11]:
p19(Meeus.dayofweek)

171

In [12]:
p19(Conway.dayofweek)

171

In [13]:
p19(Carroll.dayofweek)

171

In [14]:
p19(SysLib.dayofweek)

171